# Patient Data Query

This notebook queries patient data from multiple databases for a given prontuario.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_embrioes_congelados
- **Finops Silver Layer** (huntington_data_lake.duckdb):
  - silver.mesclada_vendas


In [1]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Input: Patient Prontuario


In [2]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Database Connections


In [3]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
        # if not df.empty:
        #     display(df)
        # else:
        #     print("No records found.")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [4]:
def query_biopsy_pgta_timeline(conn, prontuario):
    """Query biopsy and PGT-A timeline data for a specific prontuario"""
    print(f"\n=== BIOPSY PGT-A TIMELINE ===")
    query = f"""
    SELECT * FROM gold.biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_billing_timeline(conn, prontuario):
    """Query billing timeline data for a specific prontuario"""
    print(f"\n=== BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [5]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_congelamentos_semen(conn, prontuario):
    """Query view_congelamentos_semen for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_SEMEN ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_semen 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_semen")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_mesclada_vendas(conn, prontuario):
    """Query silver.mesclada_vendas for a specific prontuario"""
    print(f"\n=== SILVER.MESCLADA_VENDAS ===")
    query = f"""
    SELECT * FROM silver.mesclada_vendas 
    WHERE prontuario = {prontuario}
    ORDER BY "DT Emissao" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.mesclada_vendas")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [6]:
def query_comprehensive_timeline(conn, prontuario):
    """Query comprehensive timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [7]:
def query_resumed_timeline(conn, prontuario):
    """Query resumed timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_biopsy_pgta_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [8]:
def query_embryoscope_timeline(conn, prontuario):
    """Query embryoscope timeline data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_biopsy_pgta_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_embryoscope_timeline(conn, prontuario):
    """Query resumed embryoscope timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED EMBRYOSCOPE TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_embryoscope_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_embryoscope_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_treatment (conn, prontuario):
    """Query embryoscope treatment data for a specific prontuario"""
    print(f"\n=== EMBRYOSCOPE TREATMENT ===")
    query = f"""
    SELECT 
        patient_PatientIDx
        ,	patient_PatientID
        ,	patient_FirstName	
        , patient_LastName	
        , patient_unit_huntington	
        , embryo_KIDDate
        , embryo_FertilizationTime 
        , treatment_TreatmentName	
        , patient_unit_huntington 
    FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryoscope_embrioes")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_freeze_timeline(conn, prontuario):
    """Query embryo freeze timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_freeze_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_freeze_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_embryo_billing_timeline(conn, prontuario):
    """Query embryo billing timeline data for a specific prontuario"""
    print(f"\n=== EMBRYO BILLING TIMELINE ===")
    query = f"""
    SELECT * FROM gold.embryo_billing_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in embryo_billing_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_comprehensive_embryo_timeline(conn, prontuario):
    """Query comprehensive embryo timeline data for a specific prontuario"""
    print(f"\n=== COMPREHENSIVE EMBRYO FREEZE TIMELINE ===")
    query = f"""
    SELECT * FROM gold.comprehensive_embryo_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in comprehensive_embryo_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_cryopreservation_events_timeline(conn, prontuario):
    """Query cryopreservation events timeline data for a specific prontuario"""
    print(f"\n=== CRYOPRESERVATION EVENTS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.resumed_cryopreservation_events_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_cryopreservation_events_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_consultas_timeline(conn, prontuario):
    """Query consultation timeline data for a specific prontuario"""
    print(f"\n=== CONSULTAS TIMELINE ===")
    query = f"""
    SELECT * FROM gold.consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df

def query_resumed_consultas_timeline(conn, prontuario):
    """Query resumed consultation timeline data for a specific prontuario (most recent record only)"""
    print(f"\n=== RESUMED CONSULTAS TIMELINE (MOST RECENT RECORD) ===")
    query = f"""
    SELECT * FROM gold.resumed_consultas_timeline 
    WHERE prontuario = {prontuario}
    ORDER BY period_month DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


def get_finops_data(conn, prontuario):
    """Get finops data for a specific patient"""
    print(f"\n=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===")

    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {prontuario}
    """

    df = conn.execute(query).df()
    print(f"Found {len(df)} records in resumed_consultas_timeline")
    # if not df.empty:
    #     display(df)
    # else:
    #     print("No records found.")
    return df


In [9]:
# Updated query function with biopsy PGT-A timeline and cryopreservation data
def query_patient_data_with_timeline(prontuario):
    """
    Query all patient data including timeline data from gold layer tables.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results including timeline data
    """
    print(f"\n{'='*80}")
    print(f"QUERYING DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_embrioes = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_congelamentos_semen = query_congelamentos_semen(clinisys_con, prontuario)
        
        # Query mesclada table
        df_mesclada = query_mesclada_vendas(huntington_con, prontuario)

        # query Tratamentos timeline
        df_tratamentos_timeline = get_finops_data(huntington_con, prontuario)
        # df_billing = query_billing_timeline(huntington_con, prontuario)
        df_comprehensive = query_comprehensive_timeline(huntington_con, prontuario)
        df_resumed = query_resumed_timeline(huntington_con, prontuario)
        
        
        # Query biopsy PGT-A timeline
        df_biopsy_pgta = query_biopsy_pgta_timeline(huntington_con, prontuario)
        
        
        
        # Query embryoscope timeline
        df_embryoscope_timeline = query_embryoscope_timeline(huntington_con, prontuario)
        df_resumed_embryoscope = query_resumed_embryoscope_timeline(huntington_con, prontuario)
        
        # Query embryoscope usage
        df_embryoscope = query_embryo_treatment(huntington_con, prontuario)
        
        # Query cryopreservation timeline data
        df_embryo_freeze_timeline = query_embryo_freeze_timeline(huntington_con, prontuario)
        df_embryo_billing_timeline = query_embryo_billing_timeline(huntington_con, prontuario)
        df_comprehensive_embryo_timeline = query_comprehensive_embryo_timeline(huntington_con, prontuario)
        
        
        df_cryopreservation_events_timeline = query_cryopreservation_events_timeline(huntington_con, prontuario)
        df_resumed_cryopreservation_events_timeline = query_resumed_cryopreservation_events_timeline(huntington_con, prontuario)

        # Query consultation timeline data
        df_consultas_timeline = query_consultas_timeline(huntington_con, prontuario)
        df_resumed_consultas_timeline = query_resumed_consultas_timeline(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_congelamentos_semen: {len(df_congelamentos_semen)} records")
        print(f"- silver.mesclada_vendas: {len(df_mesclada)} records")
        print(f"- tratamentos_timeline: {len(df_tratamentos_timeline)} records")
        print(f"- biopsy_pgta_timeline: {len(df_biopsy_pgta)} records")
        # print(f"- billing_timeline: {len(df_billing)} records")
        print(f"- comprehensive_timeline: {len(df_comprehensive)} records")
        print(f"- resumed_timeline: {len(df_resumed)} records")
        print(f"- embryoscope_timeline: {len(df_embryoscope_timeline)} records")
        print(f"- resumed_embryoscope_timeline: {len(df_resumed_embryoscope)} records")
        print(f"- embryoscope_treatment: {len(df_embryoscope)} records")
        print(f"- embryo_freeze_timeline: {len(df_embryo_freeze_timeline)} records")
        print(f"- embryo_billing_timeline: {len(df_embryo_billing_timeline)} records")
        print(f"- comprehensive_embryo_timeline: {len(df_comprehensive_embryo_timeline)} records")
        print(f"- cryopreservation_events_timeline: {len(df_cryopreservation_events_timeline)} records")
        print(f"- resumed_cryopreservation_events_timeline: {len(df_resumed_cryopreservation_events_timeline)} records")
        print(f"- consultas_timeline: {len(df_consultas_timeline)} records")
        print(f"- resumed_consultas_timeline: {len(df_resumed_consultas_timeline)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'embrioes_congelados': df_embrioes,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'congelamentos_semen': df_congelamentos_semen,
            'mesclada_vendas': df_mesclada,
            'tratamentos_timeline': df_tratamentos_timeline,
            'biopsy_pgta_timeline': df_biopsy_pgta,
            # 'billing_timeline': df_billing,
            'comprehensive_timeline': df_comprehensive,
            'resumed_timeline': df_resumed,
            'embryoscope_timeline': df_embryoscope_timeline,
            'resumed_embryoscope_timeline': df_resumed_embryoscope,
            'embryoscope_treatment': df_embryoscope,
            'embryo_freeze_timeline': df_embryo_freeze_timeline,
            'embryo_billing_timeline': df_embryo_billing_timeline,
            'comprehensive_embryo_timeline': df_comprehensive_embryo_timeline,
            'cryopreservation_events_timeline': df_cryopreservation_events_timeline,
            'resumed_cryopreservation_events_timeline': df_resumed_cryopreservation_events_timeline,
            'consultas_timeline': df_consultas_timeline,
            'resumed_consultas_timeline': df_resumed_consultas_timeline
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Query Finops Silver Layer Table


In [10]:
## Example usage with comprehensive timeline data including cryopreservation


prontuario = '788592'
prontuario = 809722


# Query all patient data including comprehensive timeline and cryopreservation data
results_with_timeline = query_patient_data_with_timeline(prontuario)


QUERYING DATA FOR PRONTUARIO: 809722
Query executed at: 2025-12-15 10:10:55.519849

=== VIEW_MICROMANIPULACAO ===
Found 2 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 32 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 4 records in view_tratamentos

=== VIEW_EMBRIOES_CONGELADOS ===
Found 12 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 2 records in view_congelamentos_embrioes

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 0 records in view_congelamentos_ovulos

=== VIEW_CONGELAMENTOS_SEMEN ===
Found 0 records in view_congelamentos_semen

=== SILVER.MESCLADA_VENDAS ===
Found 48 records in silver.mesclada_vendas

=== RESUMED TRATAMENTOS TIMELINE (MOST RECENT RECORD) ===
Found 1 records in resumed_consultas_timeline

=== COMPREHENSIVE BIOPSY PGT-A TIMELINE TIMELINE ===
Found 1 records in comprehensive_biopsy_pgta_timeline

=== RESUMED BIOPSY PGT-A TIMELINE TIMELINE (MOST RECENT RECORD) ===
Found 1 records

# 0. Pagamentos

In [11]:
# Billing data
print("=== BILLING DATA ===")
mesclada_data = results_with_timeline['mesclada_vendas']
print(f"Records: {len(mesclada_data)}")
if not mesclada_data.empty:
    display(mesclada_data)
else:
    print("No billing data found.")


=== BILLING DATA ===
Records: 48


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-05-27,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10101,502,201994,RPS,200673,0,CRIOPRESERVACAO,868578,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,284226,20251209_124205,Mescladas2 (8).xlsx
1,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-05-02,"OESTROGEL PUMP GEL 0,6MG/GC/80G (C.E.)",1.0,80.0,Estimulação,10101,502,197828,RPS,196517,73,MICHELE PANZAN,868578,,000000000000531,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,279733,20251209_124205,Mescladas2 (8).xlsx
2,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-05-02,UTROGESTAN 200MG C 42 CAPSULAS (C.E.),1.0,180.0,Estimulação,10101,502,197828,RPS,196517,73,MICHELE PANZAN,868578,,000000000001684,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,279734,20251209_124205,Mescladas2 (8).xlsx
3,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-05-02,UTROGESTAN 200MG C 14 CAPSULAS (C.E.),1.0,79.0,Estimulação,10101,502,197828,RPS,196517,73,MICHELE PANZAN,868578,,000000000004632,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,279735,20251209_124205,Mescladas2 (8).xlsx
4,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.0,Criopreservação,10101,502,195504,RPS,194196,0,CRIOPRESERVACAO,868578,,S3566.39,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,277137,20251209_124205,Mescladas2 (8).xlsx
5,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-02,DUPHASTON 10MG C/ 28 COMP. REVESTIDOS (C,1.0,75.0,Estimulação,10101,502,193838,RPS,192532,73,MICHELE PANZAN,868577,,000000000004608,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,275296,20251209_124205,Mescladas2 (8).xlsx
6,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-02,"OESTROGEL PUMP GEL 0,6MG/GC/80G (C.E.)",1.0,75.0,Estimulação,10101,502,193838,RPS,192532,73,MICHELE PANZAN,868577,,000000000000531,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,275297,20251209_124205,Mescladas2 (8).xlsx
7,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-02,"LECTRUM 3,75MG (C.E.)",1.0,300.0,Estimulação,10101,502,193838,RPS,192532,73,MICHELE PANZAN,868577,,000000000004663,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,275298,20251209_124205,Mescladas2 (8).xlsx
8,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-02,UTROGESTAN 200MG C 14 CAPSULAS (C.E.),2.0,140.0,Estimulação,10101,502,193838,RPS,192532,73,MICHELE PANZAN,868577,,000000000004632,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,275299,20251209_124205,Mescladas2 (8).xlsx
9,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2024-04-02,PRIMOGYNA 2MG C 28 CP (C.E.),2.0,200.0,Estimulação,10101,502,193838,RPS,192532,73,MICHELE PANZAN,868577,,000000000004645,0.0,0.0,0.0,0.0,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,275300,20251209_124205,Mescladas2 (8).xlsx


# 1. Tratamentos

In [12]:
# Treatments timeline data
print("=== TREATMENTS TIMELINE DATA ===")
tratamentos_timeline_data = results_with_timeline['tratamentos_timeline']
print(f"Records: {len(tratamentos_timeline_data)}")
if not tratamentos_timeline_data.empty:
    display(tratamentos_timeline_data)
else:
    print("No treatments timeline data found.")


=== TREATMENTS TIMELINE DATA ===
Records: 1


,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,medico_nome,timeline_unidade,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments
0,809722,1.0,1.0,2.0,2.0,52810.0,2023-09-19,2024-05-01,2023-09-16,2024-02-09,809722,Michele Quaranta Panzan,1. HTT SP - Ibirapuera,0,0,0,0,0


In [13]:
# Treatments data
print("=== TREATMENTS DATA ===")
tratamentos_data = results_with_timeline['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No treatments data found.")


=== TREATMENTS DATA ===
Records: 4


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,prontuario_genitores,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,26455,809722,1,39,48,esposa,4,2024-05-01,Ciclo de Congelados,None,None,Hormonal,None,None,NaN,None,None,None,Homólogo,Não,NaN,None,None,PGT-A,Não,None,None,None,None,None,None,74.0,1.64,None,2024-04-14,82.0,1.75,None,Endometriose,None,None,2024-04-16,16,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,5,2024-05-01,None,None,None,Gestação Clínica,Não,None,None,None,None,None,None,None,None,None,3703,27,4bb1f4cebe437378c09a8423762ca0b4,2025-07-21 21:38:38
1,24898,809722,1,39,48,esposa,3,NaT,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,None,PGT-A,None,None,None,None,None,None,None,74.0,1.64,None,2024-01-25,82.0,1.75,None,Endometriose,None,None,2024-01-26,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Criopreservação total,No transfer,Não,None,None,None,None,None,None,None,None,None,3548,27,bbaccdb63d35028340b4e62e82a67b61,2025-07-21 21:38:38
2,22835,809722,1,39,48,esposa,2,2023-09-19,Ciclo de Congelados,None,None,Hormonal,None,None,NaN,None,None,None,Homólogo,Não,NaN,None,None,PGT-A,Não,Não,Não,None,None,None,None,74.0,1.64,None,2023-10-29,82.0,1.75,None,Endometriose,None,None,2023-10-31,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Ausência de embriões geneticamente normais,No transfer,Não,None,None,None,None,None,None,None,None,None,3703,27,23ba254807cff66164e3fd3ac8c1b81f,2025-07-21 21:38:38
3,22307,809722,1,39,48,esposa,1,NaT,Ciclo a Fresco FIV,Convencional,Antagonista,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,Não,Não,Não,None,None,None,None,74.0,1.64,Não,2023-09-06,82.0,1.75,Não,Endometriose,None,None,2023-09-07,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,None,NaT,None,None,None,No transfer,Não,0,0,0,0,0,0,0,0,None,3703,27,abb4b176a5c65493f18f575d03b16c02,2025-07-21 21:38:38


In [14]:
# Billing data
print("=== BILLING DATA TREATMENTS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Coleta - Adicional', 
    'Coleta - Crio', 
    'FET / FOT', 
    'Pró-Fiv', 
    'FIV', 
    'Inseminação', 
    'Ovodoação', 
    'Embriodoação'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TREATMENTS===
Records: 2


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
15,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-02-09,FIV - MEDICOS INTERNOS - 2º CICLO,1.0,25370.0,FIV,10101,502,186147,RPS,184884,73,MICHELE PANZAN,868578,,S3551.11,0.0,0.0,0.0,3460.0,Ibirapuera,0,0,31110100003,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,266765,20251209_124205,Mescladas2 (8).xlsx
38,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2023-09-16,FIV - MEDICOS INTERNOS,1.0,27440.0,FIV,10101,502,165927,RPS,164851,73,MICHELE PANZAN,868578,,S3551,0.0,0.0,0.0,5.0,Ibirapuera,0,0,31110100005,,,FIV,1,0,1,PRESTACAO DE SERVICO,None,NaT,False,204304,20251209_124205,Mescladas2 (8).xlsx


# 2. Biópsia - PGT-A

In [15]:
# Comprehensive timeline data
print("=== COMPREHENSIVE TIMELINE DATA ===")
comprehensive_timeline_data = results_with_timeline['comprehensive_timeline']
print(f"Records: {len(comprehensive_timeline_data)}")
if not comprehensive_timeline_data.empty:
    display(comprehensive_timeline_data)
else:
    print("No comprehensive timeline data found.")


=== COMPREHENSIVE TIMELINE DATA ===
Records: 1


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,809722,2024-02,7,7.0,13160.0,1,7.0,7.0,0.0,7,7.0,7700.0,1,7.0,7.0,0.0


In [16]:
# Resumed timeline data
print("=== RESUMED TIMELINE DATA ===")
resumed_timeline_data = results_with_timeline['resumed_timeline']
print(f"Records: {len(resumed_timeline_data)}")
if not resumed_timeline_data.empty:
    display(resumed_timeline_data)
else:
    print("No resumed timeline data found.")


=== RESUMED TIMELINE DATA ===
Records: 1


,prontuario,period_month,biopsy_count,cumulative_biopsy_count,biopsy_payment_amount,biopsy_payment_count,biopsy_payment_qtd,cumulative_biopsy_payment_qtd,biopsy_missing_payment,pgta_test_count,cumulative_pgta_test_count,pgta_payment_amount,pgta_payment_count,pgta_payment_qtd,cumulative_pgta_payment_qtd,pgta_missing_payment
0,809722,2024-02,7,7.0,13160.0,1,7.0,7.0,0.0,7,7.0,7700.0,1,7.0,7.0,0.0


In [17]:
# Billing data
print("=== BILLING DATA BIOPSY===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Biópsia Embrionária',
    'PGT-A'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA BIOPSY===
Records: 4


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
12,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-02-16,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,7.0,13160.0,Biópsia Embrionária,10101,502,187052,RPS,185791,73,MICHELE PANZAN,868578,,S3563.09,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,267764,20251209_124205,Mescladas2 (8).xlsx
13,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-02-16,ANALISE DE EMBRIOES PGT-A,7.0,7700.0,PGT-A,10101,502,187052,RPS,185791,73,MICHELE PANZAN,868578,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100024,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,267765,20251209_124205,Mescladas2 (8).xlsx
34,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2023-09-29,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,5.0,8950.0,Biópsia Embrionária,10101,502,168154,RPS,167074,73,MICHELE PANZAN,868577,,S3563.09,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,206783,20251209_124205,Mescladas2 (8).xlsx
35,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2023-09-29,ANALISE DE EMBRIOES PGT-A,5.0,4400.0,PGT-A,10101,502,168154,RPS,167074,73,MICHELE PANZAN,868577,,S3563.30,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100024,,,PGT-A,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,206784,20251209_124205,Mescladas2 (8).xlsx


In [18]:
# Micromanipulation oocytes data
print("=== MICROMANIPULATION OOCYTES DATA ===")
oocitos_data = results_with_timeline['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulation oocytes data found.")


=== MICROMANIPULATION OOCYTES DATA ===
Records: 32


,id,id_micromanipulacao,diaseguinte,Maturidade,Embriologista,PI,TCD,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,Compactando_D6,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,163715,17391,Não,MII,4321,None,Descartado,None,None,None,1CP,C1,Fresco,Não,Fresco,ICSI,Não,None,None,None,None,None,None,None,None,None,6491894f537ddd8c224a6cc76b7c5c10,2025-07-21 21:40:02,1,1
1,163716,17391,Não,MII,4321,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,11,2,Simétricos,Sim,Blastocisto Grau 4,A,A,None,None,fa8111525d7510baaecfed25c8ca8605,2025-07-21 21:40:02,1,2
2,163717,17391,Não,MII,4321,None,Descartado,None,None,None,1CP,C1,Fresco,Não,Fresco,ICSI,Não,None,None,None,None,None,None,None,None,None,160bdc2336c0d1320eb45d67c890697a,2025-07-21 21:40:02,1,3
3,163718,17391,Não,MII,4321,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,11,1,Simétricos,Sim,Blastocisto Grau 4,B,A,None,None,eae388f17dfce13d6b8a0548ede27948,2025-07-21 21:40:02,1,4
4,163719,17391,Não,MII,4321,3692,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,9,1,Simétricos,Sim,Blastocisto Grau 1,None,None,Sim,Blastocisto Grau 4,1824a06b5712cafbc04847164e30e8f3,2025-07-21 21:40:02,1,5
5,163720,17391,Não,MII,4321,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,11,2,Simétricos,Sim,Mórula B,None,None,Não,None,f82abe0aa3d993133f1e203130a30195,2025-07-21 21:40:02,1,6
6,163721,17391,Não,MII,4321,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,13,1,Simétricos,Sim,Blastocisto Grau 4,B,C,None,None,aa9612afec94864f5e5a11a8c4fd9549,2025-07-21 21:40:02,1,7
7,163722,17391,Não,MII,4321,None,Descartado,Sim,Alterado,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,10,1,Simétricos,Sim,Blastocisto Grau 1,None,None,Não,None,58e374277b1fade71fe29dcbb08d8ba1,2025-07-21 21:40:02,1,8
8,163723,17391,Não,MII,4321,3548,Criopreservado,Sim,Alterado,PGT-A,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,8,1,Simétricos,Sim,Blastocisto Grau 3,C,B,None,None,84880155d65f2116101e9345d0c042e7,2025-07-21 21:40:02,1,9
9,163724,17391,Não,MI,None,None,Descartado,None,None,None,None,C1,None,Não,Fresco,None,None,None,None,None,None,None,None,None,None,None,80f4752b71109a52b6da254eeb556128,2025-07-21 21:40:02,0,10


In [19]:
# Frozen embryos data
print("=== FROZEN EMBRYOS DATA ===")
embrioes_data = results_with_timeline['embrioes_congelados']
print(f"Records: {len(embrioes_data)}")
if not embrioes_data.empty:
    display(embrioes_data)
else:
    print("No frozen embryos data found.")


=== FROZEN EMBRYOS DATA ===
Records: 12


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,69691,183564,24584,0,809722,P8,E20716,#8,Blastocisto Grau 4 - A - B,None,None,None,None,None,None,Descartado,None,Anormal/Aneuplóide Trissomia 21 Masculino,Sim,Alterado,6.2,D5,1.24,None,None,None,e005701a265f2fe24315763b6e0b38cd,2025-07-21 21:38:57
1,69690,183563,24584,0,809722,P7,E20716,#7,Blastocisto Grau 3 - B - B,None,None,None,None,None,None,Descartado,None,"Aneuplóide complexo Monossomia 15, Trissomia 13 Feminino",Sim,Alterado,6.4,D5,7.89,None,None,None,8f1348fbf06f2766beaaf7e562146ede,2025-07-21 21:38:57
2,69689,183562,24584,0,809722,P6,E20715,#6,Blastocisto Grau 5 - A - A,None,None,None,None,None,None,Descartado,None,"Aneuplóide complexo Monossomia 22, Trissomia 21 Masculino",Sim,Alterado,8.5,D5,2.54,None,None,None,e2e886d633d386852b77ff6e4b7e97cd,2025-07-21 21:38:57
3,69688,183560,24584,14704,809722,P4,E20715,#4,Blastocisto Grau 4 - A - A,None,"EUPLOIDE, Masculino",Sim,Blastocisto Grau 4,A,A,Transferido,Sim,Normal/Euplóide Masculino,Sim,Normal,9.5,D5,8.68,None,None,None,f338b2f30d1f9654c92c9bbaecaff688,2025-07-21 21:38:57
4,69687,183559,24584,14704,809722,P3,E20715,#3,Blastocisto Grau 4 - A - A,None,"EUPLOIDE, Feminino",Sim,Blastocisto Grau 4,A,A,Transferido,Sim,Normal/Euplóide Feminino,Sim,Normal,8.4,D5,2.12,None,None,None,5c278b68059d989f803cd8cb094cf14d,2025-07-21 21:38:57
5,69686,183558,24584,0,809722,P2,E20716,#2,Blastocisto Grau 3 - A - B,None,None,None,None,None,None,Descartado,None,Aneuplóide complexo Monossomia 9 e 15 Masculino,Sim,Alterado,6.4,D5,2.67,None,None,None,02634b04a022647ecf9d365426348228,2025-07-21 21:38:57
6,69685,183557,24584,0,809722,P1,E20715,#1,Blastocisto Grau 5 - A - A,None,None,None,None,None,None,Descartado,None,Anormal/Aneuplóide Monossomia 16 Feminino,Sim,Alterado,8.8,D5,2.37,None,None,None,d000517e0d19768f14b8970f55b4f016,2025-07-21 21:38:57
7,65169,163723,23235,0,809722,P9,E20160,#9,Blastocisto Grau 3 - C - B,None,None,None,None,None,None,Descartado,None,"Aneuplóide complexo Monossomia 19, Trissomia 16 Feminino",Sim,Alterado,4.2,D5,None,None,None,None,d91564258a0ef7e6d7c2a636d2cbf316,2025-07-21 21:38:57
8,65168,163721,23235,13333,809722,P7,E20160,#7,Blastocisto Grau 4 - B - C,MANTER,"ANEUPLOIDE MONOSSOMIA 9, MASCULINO",Sim,Blastocisto Grau 4,B,C,Transferido,Sim,Anormal/Aneuplóide Monossomia 9 Masculino,Sim,Alterado,"5,0",D5,None,None,None,None,750a4b37317ae7de84abdcebf0ddefe3,2025-07-21 21:38:57
9,65167,163719,23235,0,809722,P5,E20162,#5,Blastocisto Grau 4 - B - C,None,None,None,None,None,None,Descartado,None,"Aneuplóide complexo Monossomia 15, Trissomia 11 e 19 Feminino",Sim,Alterado,3.5,D6,None,None,None,None,5690c1dd29816216d96a499b90535f59,2025-07-21 21:38:57


In [20]:
# Display each result in separate cells

# Micromanipulation data
print("=== MICROMANIPULATION DATA ===")
micromanipulacao_data = results_with_timeline['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulation data found.")


=== MICROMANIPULATION DATA ===
Records: 2


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,19310,375/24,809722,39,48,2024-02-09,11:05,11:15,07:40,27,3505,Não,10:50,08:00,08:00,08:00,None,None,19,13,Não,None,1,2024-02-09,ICSI,08 espermatozoides móveis progressivos\r\n,08 espermatozoides normais\r\n,1,0000000001,4005,5997,5997,5997,None,None,None,Laboratório,Normal,"1,5",02,03,None,None,None,None,None,None,None,None,ES 1,03,None,None,Ejaculado,08:06,09:20,"2,0",Não,"43,0","54,0",Fresco,None,1,"0,5","6,0","6,0",None,3529,Sim,0,Sim,27,Nenhum,None,None,None,esposa,8,8,0,0,8,6,0,2,0,0,0,0,0,1,0,0,0,IB192/24,None,c06a0bdfe961ffff6ed2cfae92c38e5c,2025-07-21 21:39:47
1,17391,2270/23,809722,39,48,2023-09-19,11:30,11:48,08:35,27,3652,Não,11:10,08:00,08:00,08:00,08:00,None,19,19,Não,1,1,2023-09-19,ICSI,10 espermatozoides móveis progressivos\r\n,10 espermatozoides normais\r\n,1,0000000001,4321,5997,4005,3613,None,None,None,Laboratório,Normal,"1,5",02,03,None,*Óvulo (11) foi injetado com 6h após a coleta*,None,None,None,None,None,None,ES 2,02,None,None,Ejaculado,09:01,10:11,"2,0",Não,"180,0","210,0",Fresco,None,1,"0,5","19,0","19,0",None,4003,Sim,0,Sim,27,Nenhum,*Óvulo (11) foi injetado com 6h após a coleta*,None,None,esposa,10,10,0,0,8,0,0,3,0,0,2,0,0,1,0,0,0,IB1580/23,None,8e8cda9050bf2e9cbaab5e7e335f1875,2025-07-21 21:39:47


# 3. Embryoscope

In [21]:
# Embryoscope timeline data
print("=== EMBRYOSCOPE TIMELINE DATA ===")
embryoscope_timeline_data = results_with_timeline['embryoscope_timeline']
print(f"Records: {len(embryoscope_timeline_data)}")
if not embryoscope_timeline_data.empty:
    display(embryoscope_timeline_data)
else:
    print("No embryoscope timeline data found.")


=== EMBRYOSCOPE TIMELINE DATA ===
Records: 2


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,809722,2024-02,1,2.0,1,2500.0,2.0,4270.0,0.0
1,809722,2023-09,1,1.0,1,1770.0,1.0,1770.0,0.0


In [22]:
# Resumed embryoscope timeline data
print("=== RESUMED EMBRYOSCOPE TIMELINE DATA ===")
resumed_embryoscope_data = results_with_timeline['resumed_embryoscope_timeline']
print(f"Records: {len(resumed_embryoscope_data)}")
if not resumed_embryoscope_data.empty:
    display(resumed_embryoscope_data)
else:
    print("No resumed embryoscope timeline data found.")


=== RESUMED EMBRYOSCOPE TIMELINE DATA ===
Records: 1


,prontuario,period_month,embryoscope_usage_count,cumulative_embryoscope_usage,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,embryoscope_missing_payment
0,809722,2024-02,1,2.0,1,2500.0,2.0,4270.0,0.0


In [23]:
# Billing data
print("=== BILLING DATA EMBRYOSCOPE===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Embryoscope'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA EMBRYOSCOPE===
Records: 2


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
14,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-02-09,EMBRYOSCOPE,1.0,2500.0,Embryoscope,10101,502,186145,RPS,184883,73,MICHELE PANZAN,868578,,S3563.17,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Embryoscope,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,266763,20251209_124205,Mescladas2 (8).xlsx
39,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2023-09-16,EMBRYOSCOPE,1.0,1770.0,Embryoscope,10101,502,165928,RPS,164852,73,MICHELE PANZAN,868578,,S3563.17,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100017,,,Embryoscope,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,204305,20251209_124205,Mescladas2 (8).xlsx


In [24]:
# Embryoscope treatment data
print("=== EMBRYOSCOPE TREATMENT DATA ===")
embryoscope_treatment_data = results_with_timeline['embryoscope_treatment']
print(f"Records: {len(embryoscope_treatment_data)}")
if not embryoscope_treatment_data.empty:
    display(embryoscope_treatment_data)
else:
    print("No embryoscope treatment data found.")


=== EMBRYOSCOPE TREATMENT DATA ===
Records: 20


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_unit_huntington,embryo_KIDDate,embryo_FertilizationTime,treatment_TreatmentName,patient_unit_huntington_1
0,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
1,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
2,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
3,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
4,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
5,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
6,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
7,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2024-02-14,2024-02-09 11:15:00,2024-192,Ibirapuera
8,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2023-09-25,2023-09-19 11:50:00,2023-1580,Ibirapuera
9,PC1P7BHG_45188.3833079514,809722,"PEREIRA, JULIANA A.",08/06/1984,Ibirapuera,2023-09-25,2023-09-19 11:50:00,2023-1580,Ibirapuera


# 4a. Mensalidades

In [25]:
# # Comprehensive embryo timeline data
# print("=== COMPREHENSIVE EMBRYO TIMELINE DATA ===")
# comprehensive_embryo_timeline_data = results_with_timeline['comprehensive_embryo_timeline']
# print(f"Records: {len(comprehensive_embryo_timeline_data)}")
# if not comprehensive_embryo_timeline_data.empty:
#     display(comprehensive_embryo_timeline_data)
# else:
#     print("No comprehensive embryo timeline data found.")


# 4b. Taxa Cryo

In [26]:
# Cryopreservation events timeline data
print("=== CRYOPRESERVATION EVENTS TIMELINE DATA ===")
cryopreservation_events_data = results_with_timeline['cryopreservation_events_timeline']
print(f"Records: {len(cryopreservation_events_data)}")
if not cryopreservation_events_data.empty:
    display(cryopreservation_events_data)
else:
    print("No cryopreservation events timeline data found.")


=== CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 4


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,809722,2024-03,0.0,2.0,1.0,2470.0,2.0,4820.0,0.0
1,809722,2024-02,1.0,2.0,0.0,0.0,1.0,2350.0,1.0
2,809722,2023-11,0.0,1.0,1.0,2350.0,1.0,2350.0,0.0
3,809722,2023-09,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [27]:
# Cryopreservation events timeline data
print("=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===")
resumed_cryopreservation_events_data = results_with_timeline['resumed_cryopreservation_events_timeline']
print(f"Records: {len(resumed_cryopreservation_events_data)}")
if not resumed_cryopreservation_events_data.empty:
    display(resumed_cryopreservation_events_data)
else:
    print("No resumed cryopreservation events timeline data found.")

=== RESUMED CRYOPRESERVATION EVENTS TIMELINE DATA ===
Records: 1


,prontuario,period_month,freezing_events_count,cumulative_freezing_events,billing_events_count,total_billing_amount,cumulative_billing_events,cumulative_billing_amount,events_missing_payment
0,809722,2024-03,0.0,2.0,1.0,2470.0,2.0,4820.0,0.0


In [28]:
# Billing data
print("=== BILLING DATA TAXA CRYO===")

condicao = (mesclada_data['Descricao'].str.startswith('CRIOPRESERVACAO')) | (mesclada_data['Descrição Gerencial']=='Coleta - Crio')

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA TAXA CRYO===
Records: 2


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
10,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2024-03-14,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2470.0,Criopreservação,10101,502,190906,RPS,189629,0,CRIOPRESERVACAO,868578,,S3566.04,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,272045,20251209_124205,Mescladas2 (8).xlsx
31,809723,PAULO ROBERTO,809722,JULIANA APARECIDA PEREIRA,809722,2023-11-01,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2350.0,Criopreservação,10101,502,172475,RPS,171402,0,CRIOPRESERVACAO,868578,,S3566.04,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,211931,20251209_124205,Mescladas2 (8).xlsx


In [29]:
# Embryo freezing data
print("=== EMBRYO FREEZING DATA ===")
congelamentos_embrioes_data = results_with_timeline['congelamentos_embrioes']
print(f"Records: {len(congelamentos_embrioes_data)}")
if not congelamentos_embrioes_data.empty:
    display(congelamentos_embrioes_data)
else:
    print("No embryo freezing data found.")


=== EMBRYO FREEZING DATA ===
Records: 2


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,24584,E361/24,1,809722,esposa,2024-02-14,1900-01-01 14:30:00,375/24,Não,7,7,E20715,OMEGA,4,None,Vitrificação,1.0,NaN,None,Beatriz Aiello,Priscila Fabiane Dos Santos Beirigo Paternostro,None,2ae5ab16c0f42becc39ae6d1c6dcbdc4,2025-07-21 21:38:47
1,23235,E20901/23,1,809722,esposa,2023-09-24,1900-01-01 11:00:00,2270/23,Não,5,5,E20160,AH,1,None,Vitrificação,1.0,1.0,None,Anaíde Silva Sousa,Dayane Reis,None,e9fa20a6b251357a7f7657fcb16a0c3e,2025-07-21 21:38:47


In [30]:
# Oocyte freezing data
print("=== OOCYTE FREEZING DATA ===")
congelamentos_ovulos_data = results_with_timeline['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No oocyte freezing data found.")


=== OOCYTE FREEZING DATA ===
Records: 0
No oocyte freezing data found.


In [31]:
# Semen freezing data
print("=== SEMEN FREEZING DATA ===")
congelamentos_semen_data = results_with_timeline['congelamentos_semen']
print(f"Records: {len(congelamentos_semen_data)}")
if not congelamentos_semen_data.empty:
    display(congelamentos_semen_data)
else:
    print("No semen freezing data found.")


=== SEMEN FREEZING DATA ===
Records: 0
No semen freezing data found.


# 5 Consultas

In [32]:
# Consultations timeline data
print("=== CONSULTATIONS TIMELINE DATA ===")
consultas_timeline_data = results_with_timeline['consultas_timeline']
print(f"Records: {len(consultas_timeline_data)}")
if not consultas_timeline_data.empty:
    display(consultas_timeline_data)
else:
    print("No consultations timeline data found.")


=== CONSULTATIONS TIMELINE DATA ===
Records: 2


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,809722.0,2023-10,1,2.0,0,0.0,0.0,1.0,920.0,1.0,1.0
1,809722.0,2023-06,1,1.0,1,920.0,1.0,1.0,920.0,1.0,0.0


In [33]:
# Resumed consultations timeline data
print("=== RESUMED CONSULTATIONS TIMELINE DATA ===")
resumed_consultas_data = results_with_timeline['resumed_consultas_timeline']
print(f"Records: {len(resumed_consultas_data)}")
if not resumed_consultas_data.empty:
    display(resumed_consultas_data)
else:
    print("No resumed consultations timeline data found.")


=== RESUMED CONSULTATIONS TIMELINE DATA ===
Records: 1


,prontuario,period_month,consultas_events_count,cumulative_consultas_events,billing_events_count,total_billing_amount,total_quantity,cumulative_billing_events,cumulative_billing_amount,cumulative_quantity,consultas_missing_payment
0,809722.0,2023-10,1,2.0,0,0.0,0.0,1.0,920.0,1.0,1.0


In [34]:
# Billing data
print("=== BILLING DATA cCONSULTAS===")

condicao = mesclada_data['Descrição Gerencial'].isin([
    'Consultas de CRH',
    'Outras Consultas'
])

print(f"Records: {len(mesclada_data[condicao])}")
mesclada_data[condicao]

=== BILLING DATA cCONSULTAS===
Records: 1


,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
47,809722,JULIANA APAREC,809722,JULIANA APARECIDA PEREIRA,809722,2023-06-20,PRIMEIRA CONSULTA REPRODUCAO HUMANA,1.0,920.0,Consultas de CRH,10101,502,154754,RPS,153745,73,MICHELE PANZAN,868577,,S3555.01,0.0,0.0,0.0,0.0,Ibirapuera,0,0,31110100009,,,1ª Consulta,0,0,1,PRESTACAO DE SERVICO,88,2023-09-16,True,159217,20251209_124205,Mescladas2 (8).xlsx


In [35]:
# Example: Query multiple prontuarios
# prontuarios = ['160751', '171805', '876950']
# all_results = {}
# 
# for pront in prontuarios:
#     print(f"\nQuerying prontuario: {pront}")
#     all_results[pront] = query_patient_data(pront)
# 
# # Compare results
# for pront, results in all_results.items():
#     print(f"Prontuario {pront}: {len(results['tratamentos'])} tratamentos")
